In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [21]:
#Create empty lists to store the data we are about to collect from the website
#We would like to collect the names of the cars, the mileage, the price, the rating, the number of reviews the car received

car_name = []
car_mileage = []
car_price = []

In [22]:
#We need to create a variable to store the url of the website, I'm calling this variable website_url. We then need to send a request to the website
#and scrape multiple pages

for i in range(1,11):
    website_url ="https://www.truecar.com/used-cars-for-sale/listings/location-boston-ma/?mmt[]=toyota_tacoma&mmt[]=toyota_4runner&page=" + str(i)
    #make request to the website
    web_request = requests.get(website_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'})

    # Create an object to store the html elements of the request
    soup = BeautifulSoup(web_request.content,'html.parser')

    # Create a list that contains the results of the web page
    results_list = soup.find_all('div', {'class':'mt-1 flex w-full items-start p-2 text-left flex-grow'})

    for i in results_list:

        #get the names of the cars. If the name of the car is not present, append 'n/a' to the list
        try:
            car_name.append(i.find('h2', {'data-test': 'vehicleCardInfo'}).get_text())
        except:
            car_name.append('n/a')

        #get the mileage of the cars
        try:
            car_mileage.append(i.find('div', {'data-test': 'vehicleMileage'}).get_text())
        except:
            car_mileage.append('n/a')

        #get the price of the car
        try:
            car_price.append(i.find('span', {'data-test': 'vehicleCardPricingPrice'}).get_text())
        except:
            car_price.append('n/a')

In [23]:
# Create a DataFrame

car_info = pd.DataFrame({'Name': car_name, 'Mileage': car_mileage, 'Price': car_price})

In [24]:
#print the DataFrame
car_info

,Name,Mileage,Price
0,Used 2022 ToyotaTacoma SR Double Cab 5' Bed V6...,"35,990 miles","$31,998"
1,Used 2022 Toyota4Runner SR5 4WD,"50,398 miles","$35,998"
2,Used 2023 Toyota4Runner TRD Sport 4WD,"35,682 miles","$42,990"
3,Used 2015 Toyota4Runner SR5 V6 4WD,"131,480 miles","$20,950"
4,Used 2021 Toyota4Runner Nightshade 4WD,"77,133 miles","$38,990"
...,...,...,...
304,Used 2022 ToyotaTacoma TRD Sport Double Cab 5'...,"10,631 miles","$41,595"
305,Used 2021 ToyotaTacoma SR5 Access Cab 6' Bed V...,"48,608 miles","$31,954"
306,Used 2019 ToyotaTacoma SR5 Double Cab 5' Bed V...,"64,886 miles","$29,993"
307,Used 2023 Toyota4Runner TRD Sport RWD,"28,166 miles","$40,998"


In [25]:
#Let's clean the data frame

#The first thing we should do is to edit the Reviews column

car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace('miles', '')) #remove 'miles' from the mileage column      -- placed before 'mi' so it looks for entire word first
car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace('mi', '')) #remove 'mi' from the mileage column            -- kept for sites that use 'mi' instead
car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace('k', '000')) #remove 'k' from the mileage column
car_info['Mileage'] = car_info['Mileage'].apply(lambda x:x.replace(',', '')) #remove the ',' from the mileage column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace('$', '')) # remove the $ from the Price column
car_info['Price'] = car_info['Price'].apply(lambda x:x.replace(',', '')) #remove the ',' from the Price column

In [26]:
car_info

,Name,Mileage,Price
0,Used 2022 ToyotaTacoma SR Double Cab 5' Bed V6...,35990,31998
1,Used 2022 Toyota4Runner SR5 4WD,50398,35998
2,Used 2023 Toyota4Runner TRD Sport 4WD,35682,42990
3,Used 2015 Toyota4Runner SR5 V6 4WD,131480,20950
4,Used 2021 Toyota4Runner Nightshade 4WD,77133,38990
...,...,...,...
304,Used 2022 ToyotaTacoma TRD Sport Double Cab 5'...,10631,41595
305,Used 2021 ToyotaTacoma SR5 Access Cab 6' Bed V...,48608,31954
306,Used 2019 ToyotaTacoma SR5 Double Cab 5' Bed V...,64886,29993
307,Used 2023 Toyota4Runner TRD Sport RWD,28166,40998


In [27]:
#The Name column has a lot of important data on the Year, Company, Vehicle make and model.
# Use regex to split the 'Name' column into 4 parts: Stock type, Year, Brand, and the rest
#(\w+): Matches the stock type (e.g., "Used").
#(\d{4}): Matches the year (e.g., "2021").
#([A-Za-z]+): Matches the brand (e.g., "Toyota").
#(.*): Captures the rest of the string (e.g., "Tacoma SR5 Double Cab 5' Bed V6 4WD Automatic").

car_info[['Stock type', 'Year', 'Brand', 'Rest']] = car_info['Name'].str.extract(r'(\w+)\s(\d{4})\s([A-Za-z]+)(.*)')


In [28]:
car_info

,Name,Mileage,Price,Stock type,Year,Brand,Rest
0,Used 2022 ToyotaTacoma SR Double Cab 5' Bed V6...,35990,31998,Used,2022,ToyotaTacoma,SR Double Cab 5' Bed V6 4WD Automatic
1,Used 2022 Toyota4Runner SR5 4WD,50398,35998,Used,2022,Toyota,4Runner SR5 4WD
2,Used 2023 Toyota4Runner TRD Sport 4WD,35682,42990,Used,2023,Toyota,4Runner TRD Sport 4WD
3,Used 2015 Toyota4Runner SR5 V6 4WD,131480,20950,Used,2015,Toyota,4Runner SR5 V6 4WD
4,Used 2021 Toyota4Runner Nightshade 4WD,77133,38990,Used,2021,Toyota,4Runner Nightshade 4WD
...,...,...,...,...,...,...,...
304,Used 2022 ToyotaTacoma TRD Sport Double Cab 5'...,10631,41595,Used,2022,ToyotaTacoma,TRD Sport Double Cab 5' Bed V6 4WD Automatic
305,Used 2021 ToyotaTacoma SR5 Access Cab 6' Bed V...,48608,31954,Used,2021,ToyotaTacoma,SR5 Access Cab 6' Bed V6 4WD Automatic
306,Used 2019 ToyotaTacoma SR5 Double Cab 5' Bed V...,64886,29993,Used,2019,ToyotaTacoma,SR5 Double Cab 5' Bed V6 4WD Automatic
307,Used 2023 Toyota4Runner TRD Sport RWD,28166,40998,Used,2023,Toyota,4Runner TRD Sport RWD


In [29]:
# For my example, the car name and model are displayed in the brand column. I will use regex to extract the Brand and Model
#NOTE: This may not be the case for you, if it isn't so do not run this cell. I suggest you look at your data

#We use the regular expression r'([A-Za-z]+)([A-Z][a-zA-Z]*)' to capture:
#([A-Za-z]+): The first part of the string (brand), which is made up of any sequence of letters.
#([A-Z][a-zA-Z]*): The second part (model), which starts with a capital letter followed by more letters.

car_info[['Make', 'Model']] = car_info['Brand'].str.extract(r'([A-Za-z]+)([A-Z][a-zA-Z]*)')

car_info

,Name,Mileage,Price,Stock type,Year,Brand,Rest,Make,Model
0,Used 2022 ToyotaTacoma SR Double Cab 5' Bed V6...,35990,31998,Used,2022,ToyotaTacoma,SR Double Cab 5' Bed V6 4WD Automatic,Toyota,Tacoma
1,Used 2022 Toyota4Runner SR5 4WD,50398,35998,Used,2022,Toyota,4Runner SR5 4WD,NaN,NaN
2,Used 2023 Toyota4Runner TRD Sport 4WD,35682,42990,Used,2023,Toyota,4Runner TRD Sport 4WD,NaN,NaN
3,Used 2015 Toyota4Runner SR5 V6 4WD,131480,20950,Used,2015,Toyota,4Runner SR5 V6 4WD,NaN,NaN
4,Used 2021 Toyota4Runner Nightshade 4WD,77133,38990,Used,2021,Toyota,4Runner Nightshade 4WD,NaN,NaN
...,...,...,...,...,...,...,...,...,...
304,Used 2022 ToyotaTacoma TRD Sport Double Cab 5'...,10631,41595,Used,2022,ToyotaTacoma,TRD Sport Double Cab 5' Bed V6 4WD Automatic,Toyota,Tacoma
305,Used 2021 ToyotaTacoma SR5 Access Cab 6' Bed V...,48608,31954,Used,2021,ToyotaTacoma,SR5 Access Cab 6' Bed V6 4WD Automatic,Toyota,Tacoma
306,Used 2019 ToyotaTacoma SR5 Double Cab 5' Bed V...,64886,29993,Used,2019,ToyotaTacoma,SR5 Double Cab 5' Bed V6 4WD Automatic,Toyota,Tacoma
307,Used 2023 Toyota4Runner TRD Sport RWD,28166,40998,Used,2023,Toyota,4Runner TRD Sport RWD,NaN,NaN


In [30]:
#For my example, there are a few NaNs in the Make and Model columns, I need to replace these with the values Toyota, Tacoma

# Fill NaN values in the 'Brand' column with 'Toyota'
#car_info['Make'].fillna('Toyota', inplace=True)          -- syntax reaching EOL (end of life - basically outdated)
car_info['Make'] = car_info['Make'].fillna('Toyota')      # syntax up-to-date

# Fill NaN values in the 'Model' column with '4Runner'    -- NaN values only present in rows with 4Runner (replacing with Tacoma makes data unreliable)
#car_info['Model'].fillna('Tacoma', inplace=True)         -- also: syntax reaching EOL
car_info['Model'] = car_info['Model'].fillna('4Runner')   # syntax up-to-date

# Display the DataFrame
print(car_info)

                                                  Name  Mileage   Price  \
0    Used 2022 ToyotaTacoma SR Double Cab 5' Bed V6...   35990   31998    
1                      Used 2022 Toyota4Runner SR5 4WD   50398   35998    
2                Used 2023 Toyota4Runner TRD Sport 4WD   35682   42990    
3                   Used 2015 Toyota4Runner SR5 V6 4WD  131480   20950    
4               Used 2021 Toyota4Runner Nightshade 4WD   77133   38990    
..                                                 ...      ...     ...   
304  Used 2022 ToyotaTacoma TRD Sport Double Cab 5'...   10631   41595    
305  Used 2021 ToyotaTacoma SR5 Access Cab 6' Bed V...   48608   31954    
306  Used 2019 ToyotaTacoma SR5 Double Cab 5' Bed V...   64886   29993    
307              Used 2023 Toyota4Runner TRD Sport RWD   28166   40998    
308  Used 2022 ToyotaTacoma SR Double Cab 5' Bed V6...   48731   30998    

    Stock type  Year         Brand  \
0         Used  2022  ToyotaTacoma   
1         Used  2022   

In [31]:
# Drop the column Name and its no longer needed
car_info = car_info.drop(["Name"], axis = 1)
car_info = car_info.drop(["Rest"], axis = 1)
car_info = car_info.drop(["Brand"], axis = 1)

In [32]:
car_info

,Mileage,Price,Stock type,Year,Make,Model
0,35990,31998,Used,2022,Toyota,Tacoma
1,50398,35998,Used,2022,Toyota,4Runner
2,35682,42990,Used,2023,Toyota,4Runner
3,131480,20950,Used,2015,Toyota,4Runner
4,77133,38990,Used,2021,Toyota,4Runner
...,...,...,...,...,...,...
304,10631,41595,Used,2022,Toyota,Tacoma
305,48608,31954,Used,2021,Toyota,Tacoma
306,64886,29993,Used,2019,Toyota,Tacoma
307,28166,40998,Used,2023,Toyota,4Runner


In [35]:
#You can export the DataFrame to excel if you choose to proceed with excel
car_info.to_excel('cars-toyota-tacoma-4runner.xlsx', index = False)

In [36]:
#You can export the DataFrame to a csv file if needed
car_info.to_csv('cars-toyota-tacoma-4runner.csv', index = False)